# Problem - Create and verify the digital signature of your full name using RSA with MD5 as hash function

Submitted by Ariba Siddiqui (15BCS0007)

## 1.1 RSA key-pair generation

To generate key-pair for digital signature - RSA algorithm, most widely-used public key cryptography algorithm in the world, is used.

The algorithm is as follows:   
-  Select p, q such that p and q both are primes and p≠q. 
-  Calculate n = p x q. 
-  Calculate Φ(n)  = (p -1) x (q - 1). 
-  Select integer e such that gcd(Φ(n),e)=1 and where 1 < e < Φ(n). 
-  Calculate d = e-1 mod Φ(n). i.e. ed = 1 mod Φ(n)   
-  Public key KU = {e, n}. 
-  Private key KR = {d, n}. 


In [1]:
import random, math

'''
Euclid's extended algorithm for finding the multiplicative inverse of two numbers
'''
def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi
    
    while e > 0:
        temp1 = temp_phi//e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2
        
        x = x2- temp1* x1
        y = d - temp1 * y1
        
        x2 = x1
        x1 = x
        d = y1
        y1 = y
    
    if temp_phi == 1:
        return d + phi

'''
Euclid's algorithm for determining the greatest common divisor
Use iteration to make it faster for larger integers
'''
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

'''
Test to see if a number is prime.
'''
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True

def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    #n = pq
    n = p * q

    #Phi is the totient of n
    phi = (p-1) * (q-1)

    #Choose an integer e such that e and phi(n) are coprime
    e=2
    while math.gcd(e,phi)!=1:
        e = random.randrange(1, phi)
    print("\nThe value of e selected : ",e)
    #Use Euclid's Algorithm to verify that e and phi(n) are comprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(2, phi)
        g = gcd(e, phi)

    #Use Extended Euclid's Algorithm to generate the private key
    d = multiplicative_inverse(e, phi)
    
    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))



if __name__ == '__main__':
    '''
    Detect if the script is being run directly by the user
    '''
    print ("RSA key-pair generator")
    p = int(input("Enter a prime number (17, 19, 23, etc): "))
    q = int(input("Enter another prime number (Not one you entered above): "))
    print ("\nGenerating your public/private keypairs now . . .\n")
    public, private = generate_keypair(p, q)
    print("\nDone.\n")
    print("public key : ",public)
    print("private key : ",private)

RSA key-pair generator
Enter a prime number (17, 19, 23, etc): 7
Enter another prime number (Not one you entered above): 13

Generating your public/private keypairs now . . .


The value of e selected :  5

Done.

public key :  (5, 91)
private key :  (101, 91)


## 1.2 Sign Generation

-  Given  a  message m,  we  apply  a  suitable  hash  function  H  (MD5, SHA1 or SHA2) to obtain the hash result $ M = H(m). $
-  To sign a   message m,   we   use   M   <   n   to   compute   Signature $ S = M^d (mod n)$ where d is the private key of the signer. 



The Figure shows the data flow diagram of Signature generation module in application RSAAPP.

<img src = "yes.png">


We use __MD5 hash function__ in this problem.

### 1.2.1 MD5 hash 

The  algorithm  takes  as  input  a  message  of   arbitrary  length  and produces as output a 128-bit “fingerprint” or “message digest” of the input. It is  conjectured  that  it  is  computationally  infeasible  to  produce  two  messages having  the  same  message  digest,  or  to  produce  any  message  having  a  given prespecified  target  message  digest.  The  MD5algorithm  is  intended  for  digital signature  applications,  where  a  large  file  must  be “compressed”  in  a  secure manner before being encrypted with a private (secret) key under a public-key cryptosystem  such  as  RSA.  Figure below depicts  the  way  the  input  message  is turned into a 128-bit message digest. 
<img src = "yess.png">


Here we compute the message digest of "Ariba Siddiqui" 

In [2]:
import math
 
rotate_amounts = [7, 12, 17, 22, 7, 12, 17, 22, 7, 12, 17, 22, 7, 12, 17, 22,
                  5,  9, 14, 20, 5,  9, 14, 20, 5,  9, 14, 20, 5,  9, 14, 20,
                  4, 11, 16, 23, 4, 11, 16, 23, 4, 11, 16, 23, 4, 11, 16, 23,
                  6, 10, 15, 21, 6, 10, 15, 21, 6, 10, 15, 21, 6, 10, 15, 21]
 
constants = [int(abs(math.sin(i+1)) * 2**32) & 0xFFFFFFFF for i in range(64)]
 
init_values = [0x67452301, 0xefcdab89, 0x98badcfe, 0x10325476]
 
functions = 16*[lambda b, c, d: (b & c) | (~b & d)] + \
            16*[lambda b, c, d: (d & b) | (~d & c)] + \
            16*[lambda b, c, d: b ^ c ^ d] + \
            16*[lambda b, c, d: c ^ (b | ~d)]

index_functions = 16*[lambda i: i] + \
                  16*[lambda i: (5*i + 1)%16] + \
                  16*[lambda i: (3*i + 5)%16] + \
                  16*[lambda i: (7*i)%16]

def left_rotate(x, amount):
    x &= 0xFFFFFFFF
    return ((x<<amount) | (x>>(32-amount))) & 0xFFFFFFFF
 
def md5(message):
 
    message = bytearray(message) #copy our input into a mutable buffer
    orig_len_in_bits = (8 * len(message)) & 0xffffffffffffffff
    message.append(0x80)
    while len(message)%64 != 56:
        message.append(0)
    message += orig_len_in_bits.to_bytes(8, byteorder='little')
 
    hash_pieces = init_values[:]
 
    for chunk_ofst in range(0, len(message), 64):
        a, b, c, d = hash_pieces
        chunk = message[chunk_ofst:chunk_ofst+64]
        for i in range(64):
            f = functions[i](b, c, d)
            g = index_functions[i](i)
            to_rotate = a + f + constants[i] + int.from_bytes(chunk[4*g:4*g+4], byteorder='little')
            new_b = (b + left_rotate(to_rotate, rotate_amounts[i])) & 0xFFFFFFFF
            a, b, c, d = d, new_b, b, c
        for i, val in enumerate([a, b, c, d]):
            hash_pieces[i] += val
            hash_pieces[i] &= 0xFFFFFFFF
 
    return sum(x<<(32*i) for i, x in enumerate(hash_pieces))
 
def md5_to_hex(digest):
    raw = digest.to_bytes(16, byteorder='little')
    return '{:032x}'.format(int.from_bytes(raw, byteorder='big'))
 
if __name__=='__main__':
    message = b"Ariba Siddiqui"
    print("MD5 message digest of my name . . .")
    digest = md5_to_hex(md5(message))
    print(digest,' <= "',message.decode('ascii'),'"', sep='')
 

MD5 message digest of my name . . .
b890d7690a609ed76c6cfde430082530 <= "Ariba Siddiqui"


Comparing with the result of _md5()_ built-in function provided in _hashlib_ (just to check if the above result is right) ...

In [3]:
import hashlib
m = hashlib.md5()
m.update(b'Ariba Siddiqui')
digest = m.hexdigest()
digest

'b890d7690a609ed76c6cfde430082530'

So, we're right so far! :D

Next, we convert this 128-bit digest to an integer value mod(n).

In [4]:
M = int(digest,16)
print("The value of M (H(m))for our message is : ",M%private[1])

The value of M (H(m))for our message is :  13


### 1.2.2 Signature generation

Compute Signature, where d is private key value.. $$ S = M^d (mod n) $$

In [5]:
S = pow(M,private[0],private[1])
print("The value of S is : ",S)

The value of S is :  13


## 1.3 Sign Verification

-  To  verify  the  message m,  we  use  the  digital  signature  S  to  compute $N = S^e (mod n)$ where e is the public key of the signer. 
-  Then we obtain $M = H(m)$ and compare it with N. 
-  If  both  are  same  then  the  message  is  authentic  otherwise  it  is tempered. 

### 1.3.1 N calculation

In [6]:
N = pow(S,public[0],public[1])
print("The value of N :",N)

The value of N : 13


### 1.3.2 M calculation

In [7]:
M = int(md5_to_hex(md5(b'Ariba Siddiqui')),16)%public[1]
print("The calculated value of M is :",M)

The calculated value of M is : 13


### 1.3.3 Comparision

In [8]:
if M==N:
    print("Signature verified!!")
else:
    print("There's some bug :(")

Signature verified!!


THE END :D